<a href="https://colab.research.google.com/github/lpoggetto/fiap_recommendation_system/blob/main/trabalho_SR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Trabalho de Sistemas de Recomendação

* Cleber Veiga Galvão - RM: 356846
* Danildo André Amorim dos Santos - RM: 355182
* Victor Marcio Hayee Falcão -  RM: 354955
* Lucas Ribeiro Del Poggetto - RM: 355220

In [12]:
import requests
import random
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
from pandas import json_normalize
from time import sleep

import warnings
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# URL da API Open Finance (substitua com a URL da sua API)
urls = [
'https://api.itau/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://opendata.api.nubank.com.br/open-banking/opendata-creditcards/v1/personal-credit-cards','https://resources.openbanking.picpay.com/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://openbanking.api.santander.com.br/open-banking/opendata-creditcards/v1/personal-credit-cards',
#'https://openbanking.bib.com.br/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://api.mercadopago.com/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://opendata.api.bb.com.br/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://api-bmg.bancobmg.com.br/open-banking/opendata-creditcards/v1/personal-credit-cards',
#'https://api.bradesco.com/next/open-banking/opendata-creditcards/v1/personal-credit-cards',
# 'https://api.bradesco.com/bradesco/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://openbanking.banrisul.com.br/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://api-openbanking.bancopan.com.br/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://api.safra.com.br/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://banking-openfinance.xpi.com.br/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://banking-openfinance.xpi.com.br/rico/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://banking-openfinance-azimutbrasil.xpi.com.br/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://banking-openfinance.xpi.com.br/clear/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://banking-openfinance.xpi.com.br/modal/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://banking-openfinance-montebravo.xpi.com.br/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://banking-openfinance-whg.xpi.com.br/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://api-openbanking.bvopen.com.br/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://api.sicredi.com.br/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://api-opf.asa.com.br/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://ob-api.sisprimedobrasil.com.br/open-banking/opendata-creditcards/v1/personal-credit-cards',
# 'https://openfinance.sicoob.com.br/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://authorization.openapi.sofisa.com.br/open-banking/opendata-creditcards/v1/personal-credit-cards',
# 'https://api.openbanking.caixa.gov.br/open-banking/opendata-creditcards/v1/personal-credit-cards',
# 'https://api.bradesco.com/bradescard/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://api.e-unicred.com.br/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://apps.neon.com.br/open-finance/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://openbanking.digio.com.br/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://btgmais.openbanking.btgpactual.com/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://api.openfinance.bnb.gov.br/open-banking/opendata-creditcards/v1/personal-credit-cards'
]

# Fazendo a requisição GET à API
for url in urls:
  response = requests.get(url)

  # Verificando se a requisição foi bem-sucedida (status code 200)
  if response.status_code == 200:
    try:
      df1 = pd.json_normalize(response.json(), 'data')
      dffinal = pd.concat([dffinal, df1])
    except:
      dffinal = pd.json_normalize(response.json(), 'data')
  else:
      print(f"Falha na requisição. Código de status: {response.status_code}")


Falha na requisição. Código de status: 403


In [13]:
# selecionando apenas variaveis do estudo
cols = ['participant.brand', 'identification.product.type', 'identification.creditCard.network', 'rewardsProgram.hasRewardProgram']

dffinal_sr = dffinal[cols]

dffinal_sr['rewardsProgram.hasRewardProgram'].replace({True: 'SIM', False: 'NAO'}, inplace=True)

dffinal_sr.drop_duplicates(['participant.brand', 'identification.product.type', 'identification.creditCard.network', 'rewardsProgram.hasRewardProgram'], inplace=True)

# simulado "upvotes" de tipos de cartão de credito
dffinal_sr.loc[:, 'upvotes'] = np.random.randint(0, 100, size=len(dffinal_sr))


In [14]:
# Funcao - algoritmo recomendacao
def recommend_cards(dataset, network=None, has_rewards=None, product_type=None, recomendacoes = None):
    """
    Filtra cartões de crédito com base em critérios do usuário

    Parâmetros:
    - dataset: Seu DataFrame (dffinal)
    - network: Bandeira do cartão (VISA, Mastercard, American Express, etc.)
    - has_rewards: Booleano indicando se o cartão tem programa de recompensas (True/False)
    - brand: Nome da marca emissora
    - product_type: Tipo de produto (Cartão de Crédito, Cartão de Débito, etc.)

    Retorna:
    - DataFrame filtrado com os cartões recomendados
    """

    # Instead of using query, directly filter using boolean indexing:
    filtered_dataset = dataset

    if network:
        filtered_dataset = filtered_dataset[filtered_dataset['identification.creditCard.network'] == network]
    if has_rewards is not None:
        filtered_dataset = filtered_dataset[filtered_dataset['rewardsProgram.hasRewardProgram'] == has_rewards]
    if product_type:
        filtered_dataset = filtered_dataset[filtered_dataset['identification.product.type'] == product_type]

    return filtered_dataset[['participant.brand', 'identification.product.type', 'identification.creditCard.network', 'rewardsProgram.hasRewardProgram', 'upvotes']].sort_values('upvotes', ascending=False).head(recomendacoes)

In [15]:
## Sistema de recomendação - Quantum Finance
# @title Selecione qual o melhor cartão para você!

# banco = 'ITAU' # @param ['ITAU', 'Nubank', 'PICPAY', 'Banco Santander', 'MERCADO PAGO', 'Banco do Brasil S/A', 'Banrisul', 'Banco Pan', 'Grupo J. Safra', 'Banco XP', 'Banco XP (Rico)', 'Banco XP (Azimut)', 'banco BV', 'Sicredi', 'ASA', 'SISPRIME DO BRASIL', 'Banco Sofisa', 'UNICRED DO BRASIL', 'Neon', 'Banco Digio', 'BTG Banking', 'Banco do Nordeste do Brasil S.A.']
tipo_do_cartao = 'BLACK' # @param ['STANDARD_INTERNACIONAL', 'CLASSIC_INTERNACIONAL', 'PLATINUM', 'STANDARD_NACIONAL', 'GOLD', 'BLACK', 'INFINITE', 'CLASSIC_NACIONAL', 'ELETRONIC', 'OUTROS']
processadora = 'MASTERCARD' # @param ['MASTERCARD', 'VISA', 'AMERICAN_EXPRESS', 'ELO', 'OUTRAS']
programa_de_recompensas = 'SIM' # @param ['SIM',  'NAO']
qtd_recomendacoes = 5 # @param {type:"slider", min:1, max:5, step:1}

In [16]:
recommend_cards(dataset=dffinal_sr,
                product_type=tipo_do_cartao,
                network=processadora,
                has_rewards=programa_de_recompensas,
                recomendacoes=qtd_recomendacoes
                )

,participant.brand,identification.product.type,identification.creditCard.network,rewardsProgram.hasRewardProgram,upvotes
0,BTG Banking,BLACK,MASTERCARD,SIM,41
0,Banco Santander,BLACK,MASTERCARD,SIM,25
17,Banrisul,BLACK,MASTERCARD,SIM,25
0,Sicredi,BLACK,MASTERCARD,SIM,1
4,Grupo J. Safra,BLACK,MASTERCARD,SIM,0
